# 04_build_scenes Prompt API Playground

Scope: API-only prompt testing for `harness/prompts/04_build_scenes`.

This notebook is self-contained and does **not** invoke the pipeline or write scene files.

## Cell 1: Setup

Purpose:
- Resolve repo imports from notebook context.
- Load `.env` automatically.
- Import prompt template helpers and API client.

In [43]:
import json
import os
import sys
import tempfile
from pathlib import Path

# Resolve repo root whether cwd is repo root, notebooks/, or another child folder.
cwd = Path.cwd().resolve()
candidates = [cwd, *cwd.parents]
repo_root = None
for p in candidates:
    if (p / 'harness').exists() and (p / 'notebooks').exists():
        repo_root = p
        break
if repo_root is None:
    raise RuntimeError('Could not locate repo root containing harness/ and notebooks/')
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Load environment from repo .env if possible.
env_path = repo_root / '.env'
try:
    from dotenv import load_dotenv
    if env_path.exists():
        load_dotenv(env_path)
except Exception:
    pass

from harness.prompts import compose_prompt
from harness.client import LLMClient, estimate_tokens

print(f'Repo root: {repo_root}')



Repo root: /Users/velocityworks/IdeaProjects/flaming-horse


## Cell 2: Hardcoded Build Inputs

Purpose:
- Define `plan_data`, `narration_script`, and scene state inline.
- Select which scene to build via `current_scene_index`.

In [44]:
plan_data = {
    "title": "How Matrix Multiplication Works",
    "description": "A step-by-step guide to understanding matrix multiplication.",
    "target_duration_seconds": 360,
    "scenes": [
        {
            "id": "scene_01_intro",
            "title": "Introduction to Matrix Multiplication",
            "narration_key": "scene_01_intro",
            "description": "Introduce motivation and preview structure.",
            "estimated_duration_seconds": 25,
            "visual_ideas": [
                "Title and matrix icons",
                "A and B slide in",
                "C = A * B appears"
            ]
        },
        {
            "id": "scene_02_matrix_basics",
            "title": "What is a Matrix?",
            "narration_key": "scene_02_matrix_basics",
            "description": "Define matrix, rows/columns, notation.",
            "estimated_duration_seconds": 30,
            "visual_ideas": [
                "Grid fills with values",
                "Row/column labels",
                "Vector vs matrix comparison"
            ]
        }
    ]
}

narration_script = {
    "scene_01_intro": "Welcome to how matrix multiplication works. We'll see why it matters, then walk through the core rule and examples.",
    "scene_02_matrix_basics": "A matrix is a rectangular grid of numbers with rows and columns. We'll use dimensions and index notation to describe each element clearly."
}

def scene_id_to_class_name(scene_id: str) -> str:
    parts = [p for p in str(scene_id).split('_') if p]
    return ''.join(p.capitalize() for p in parts)

# Mirrors the state layout harness expects for build_scenes.
state = {
    "scenes": [
        {
            "id": "scene_01_intro",
            "title": "Introduction to Matrix Multiplication",
            "file": "scene_01_intro.py",
            "class_name": "Scene01Intro",
            "narration_key": "scene_01_intro"
        },
        {
            "id": "scene_02_matrix_basics",
            "title": "What is a Matrix?",
            "file": "scene_02_matrix_basics.py",
            "class_name": "Scene02MatrixBasics",
            "narration_key": "scene_02_matrix_basics"
        }
    ],
    "current_scene_index": 0
}

# Optional retry details for scene regeneration tests.
retry_context = None

temperature = 0.7
max_tokens = 16000

## Cell 3: Compose Prompts

Purpose:
- Compose `build_scenes` system/user prompts using exact harness templates.
- Fill placeholders from the in-notebook state/plan/narration data.

In [45]:
# Build a temporary project fixture and call harness.compose_prompt directly
# so this notebook always matches harness prompt behavior exactly.

with tempfile.TemporaryDirectory(prefix='fh_prompt_playground_') as td:
    project_dir = Path(td)
    (project_dir / 'plan.json').write_text(json.dumps(plan_data, indent=2), encoding='utf-8')

    narration_py = 'SCRIPT = ' + json.dumps(narration_script, indent=2) + '
'
    (project_dir / 'narration_script.py').write_text(narration_py, encoding='utf-8')

    state_for_prompt = {
        'plan_file': 'plan.json',
        'narration_file': 'narration_script.py',
        'scenes': state['scenes'],
        'current_scene_index': state['current_scene_index'],
    }

    system_prompt, user_prompt = compose_prompt(
        phase='build_scenes',
        state=state_for_prompt,
        project_dir=project_dir,
        retry_context=retry_context,
    )

scene = state['scenes'][state['current_scene_index']]
scene_id = scene['id']
scene_title = scene['title']
narration_key = scene.get('narration_key', scene_id)
scene_class_name = scene.get('class_name', '')

print(f'Current scene: {scene_id} ({scene_title})')
print(f'System prompt chars: {len(system_prompt):,}')
print(f'User prompt chars:   {len(user_prompt):,}')
print(f'System est tokens:   {estimate_tokens(system_prompt):,}')
print(f'User est tokens:     {estimate_tokens(user_prompt):,}')



Current scene: scene_01_intro (Introduction to Matrix Multiplication)
System prompt chars: 27,159
User prompt chars:   3,037
System est tokens:   6,789
User est tokens:     759


## Cell 4: Prompt Preview

Purpose:
- Inspect prompt snippets before sending to API.

In [46]:
print('=== SYSTEM PROMPT PREVIEW ===')
print(system_prompt)
print('\n=== USER PROMPT PREVIEW ===')
print(user_prompt)

=== SYSTEM PROMPT PREVIEW ===
Video Production Agent - Build Scenes Phase



System role:
You are an expert Manim programmer creating compelling animations.
Use Manim Community Edition reference docs: https://docs.manim.community/en/stable/reference.html

System objective:
Produce high-quality scene content that is semantically faithful to narration and plan intent.
Follow run-specific output format and hard requirements from the user prompt.

Configuration and positioning guide:


Visual helpers and aesthetics:


Kitchen sink reference:
# Manim CE Kitchen Sink (Static Agent Reference)

Use this document as direct implementation guidance when generating Manim scene code.
This file is designed for system-prompt injection and contains concrete examples from official Manim CE documentation.

---

## Build Scenes Output Contract (SAFE_FOR_BUILD_SCENES)

When this reference is injected into the `build_scenes` phase, follow this contract first.

- `SAFE_FOR_BUILD_SCENES`: `self.play(...)`, `

## Cell 5: API Call

Purpose:
- Send composed prompts to the LLM API and print raw response text.

In [47]:
provider = os.getenv('LLM_PROVIDER', 'XAI').upper()
api_key_var = f'{provider}_API_KEY'
assert os.getenv(api_key_var), f'Missing {api_key_var} in environment/.env'

client = LLMClient()
response_text = client.chat_completion(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    temperature=temperature,
    max_tokens=max_tokens,
)

print(response_text)

🤖 Harness using:
   Provider: XAI
   Base URL: https://api.x.ai/v1
   Model: grok-4-1-fast-reasoning
```python
title = Text("Introduction to Matrix Multiplication", font_size=48, weight=BOLD)
title.move_to(UP * 3.8)
self.play(Write(title))

a_matrix = MathTex(
    r"A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}",
    font_size=48,
    color=BLUE
).move_to(LEFT * 5 + DOWN)

b_matrix = MathTex(
    r"B = \begin{pmatrix} 5 & 6 \\ 7 & 8 \end{pmatrix}",
    font_size=48,
    color=GREEN
).move_to(RIGHT * 5 + DOWN)

c_eq = MathTex(r"C = A \times B", font_size=64, color=YELLOW).move_to(DOWN * 0.2)

with self.voiceover(text=SCRIPT["scene_01_intro"]) as tracker:
    self.play(
        a_matrix.animate.move_to(LEFT * 2 + DOWN),
        b_matrix.animate.move_to(RIGHT * 2 + DOWN),
        run_time=3
    )
    self.play(FadeIn(c_eq))
    self.wait(3)
```


## Cell 6: Response Sanity Checks

Purpose:
- Run lightweight checks on returned code text.
- No files are written.

In [48]:
checks = {
    'contains class name': scene_class_name in response_text,
    'contains VoiceoverScene': 'VoiceoverScene' in response_text,
    'contains narration key': narration_key in response_text,
    'imports get_speech_service': 'get_speech_service' in response_text,
}

for k, v in checks.items():
    print(f'{k}: {v}')

missing = [k for k, v in checks.items() if not v]
print('missing checks:', missing)

contains class name: False
contains VoiceoverScene: False
contains narration key: True
imports get_speech_service: False
missing checks: ['contains class name', 'contains VoiceoverScene', 'imports get_speech_service']
